In [1]:
import numpy as np
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
def log_sum_exp(Z):
    """ Compute log(\sum_i exp(Z_i)) for some array Z."""
    return np.max(Z) + np.log(np.sum(np.exp(Z - np.max(Z))))

def loglikelihood(data, weights, means, covs):
    """ Compute the loglikelihood of the data for a Gaussian mixture model with the given parameters. """
    num_clusters = len(means)
    num_dim = len(data[0])
    
    ll = 0
    for d in data:
        
        Z = np.zeros(num_clusters)
        for k in range(num_clusters):
            
            # Compute (x-mu)^T * Sigma^{-1} * (x-mu)
            delta = np.array(d) - means[k]
            exponent_term = np.dot(delta.T, np.dot(np.linalg.inv(covs[k]), delta))
            
            # Compute loglikelihood contribution for this data point and this cluster
            Z[k] += np.log(weights[k])
            Z[k] -= 1/2. * (num_dim * np.log(2*np.pi) + np.log(np.linalg.det(covs[k])) + exponent_term)
            
        # Increment loglikelihood contribution of this data point across all clusters
        ll += log_sum_exp(Z)
        
    return ll

In [25]:
len(means)

NameError: name 'means' is not defined

In [23]:
def EM(data, init_means, init_covariances, init_weights, maxiter=20):
    
    # Make copies of initial parameters, which we will update during each iteration
    means = init_means[:]
    covariances = init_covariances[:]
    weights = init_weights[:]
    
    # Infer dimensions of dataset and the number of clusters
    num_data = len(data)
    num_dim = len(data[0])
    num_clusters = len(means)
    
    # Initialize some useful variables
    resp = np.zeros((num_data, num_clusters))
    ll = loglikelihood(data, weights, means, covariances)
    ll_trace = [ll]
    
    for i in range(maxiter):
        # E-step: compute responsibilities
        for j in range(num_data):
            for k in range(num_clusters):
                resp[j, k] = weights[k]*multivariate_normal.pdf(data[j],means[k],covariances[k])
        row_sums = resp.sum(axis=1)[:, np.newaxis]
        resp = resp / row_sums # normalize over all possible cluster assignments

        # M-step
        counts = np.sum(resp, axis=0)
        print ("num_data:{}".format(num_data))
        for k in range(num_clusters):
            weights[k] = counts[k]/num_data
            weighted_sum = 0
            for j in range(num_data):
                weighted_sum += (resp[j,k]*data[j])
            means[k] = weighted_sum/counts[k]
            
            weighted_sum = np.zeros((num_dim, num_dim))
            for j in range(num_data):
                weighted_sum += (resp[j,k]*np.outer(data[j]-means[k],data[j]-means[k]))
            covariances[k] = weighted_sum/counts[k]
        
        # Compute the loglikelihood at this iteration
        ll_latest = loglikelihood(data, weights, means, covariances)
        ll_trace.append(ll_latest)
        
        ll = ll_latest
    
        print("Iteration : {} - means:{}".format(i,means))
        
        #plt.figure(figsize=(12,8))
        #plt.scatter(X[:,0],X[:,1])
        #plt.scatter(means[0][0], means[0][1], color = "red")
        #plt.scatter(means[1][0], means[1][1],color="orange")
    
    out = {'weights': weights, 'means': means, 'covs': covariances, 'loglik': ll_trace, 'resp': resp}

    return out

In [13]:
#Load the Data
X = np.loadtxt('data/Faithful.txt')

In [28]:
len(initial_weights)

2

In [26]:
np.random.seed(234)

#Two Cluster
initial_means = [np.array([3.467750,70.132353]),np.array([3.5078162,71.6617647])]
initial_covs = [np.array([[1.2975376,13.9110994],[13.911099,183.559040]])]*2
initial_weights = [0.50062804,0.49937196]

#Three Cluster
#initial_means = [np.array([3.4459639,69.8433735]),np.array([3.6217053,72.1578947]),np.array([3.3893617,70.5531915])]
#initial_covs = [np.array([[1.2877935,13.842302],[13.8423020,183.208932]])]*3
#initial_weights = [0.30514706,0.34926471,0.34558824]

# Run EM 
results = EM(X, initial_means, initial_covs, initial_weights)

num_data:272
Iteration : 0 - means:[array([ 3.46776969, 70.13260977]), array([ 3.5078488 , 71.66350617])]
num_data:272
Iteration : 1 - means:[array([ 3.46294816, 70.05882059]), array([ 3.51268552, 71.73757547])]
num_data:272
Iteration : 2 - means:[array([ 3.45668322, 69.95616588]), array([ 3.51897131, 71.84062497])]
num_data:272
Iteration : 3 - means:[array([ 3.44929539, 69.83095522]), array([ 3.52638636, 71.96636365])]
num_data:272
Iteration : 4 - means:[array([ 3.4406794 , 69.68368151]), array([ 3.53503842, 72.11434259])]
num_data:272
Iteration : 5 - means:[array([ 3.43064547, 69.51340357]), array([ 3.54512126, 72.28557083])]
num_data:272
Iteration : 6 - means:[array([ 3.41895944, 69.31842936]), array([ 3.55687499, 72.48184128])]
num_data:272
Iteration : 7 - means:[array([ 3.40534086, 69.09644779]), array([ 3.57058905, 72.70561393])]
num_data:272
Iteration : 8 - means:[array([ 3.38945156, 68.84452123]), array([ 3.58661524, 72.96004629])]
num_data:272
Iteration : 9 - means:[array([ 3.

In [6]:
results['weights']

[0.5036365730555193, 0.49636342694448077]

In [7]:
results['means']

[array([ 2.73291856, 61.53818513]), array([ 4.25370854, 80.39306683])]

In [8]:
results['covs'][0]

array([[  1.15708628,  11.82112052],
       [ 11.82112052, 149.17019173]])